# 📦 Import Packages and Connect Google Drive  

Before we can work with data or build AI models, we need some **tools**.  
In Python, these tools are called **packages (or libraries)**. Think of them like apps you download on your phone: each one has a special function.  

- **pandas (pd):** Like Excel in Python — helps us load and organize data tables.  
- **numpy (np):** A calculator for fast math — handles large sets of numbers.  
- **matplotlib.pyplot (plt):** A drawing tool — used to make graphs and plots.  
- **scipy.stats (sp):** A toolbox for statistics — helps with averages, probability, etc.  
- **pywt:** A tool for signal analysis — helps us break down data into patterns (wavelets).  

We need to load these tools before we can use them in the notebook.  

In [1]:
# Import basic packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sp
import pywt

### 🔗 Mount Google Drive  

Google Colab runs in the cloud, but we often store our files (like data or reports) in Google Drive.  
To make Colab read or save files directly to your Drive, we need to **“mount”** it.  
You can think of this like plugging in a USB drive so your computer can access its files.  

- When you run the code below, Colab will ask for your Google account permission.  
- After granting access, your Drive will appear in the folder `/content/drive`.  
- From now on, we can open files directly from your Drive or save results there.  

In [ ]:
# Mount your google drive
from google.colab import drive
drive.mount('/content/drive')

.

.

.

# 📂 1. Loading the Dataset from GitHub  

Now we will load the dataset that we will use for our AI project.  
This dataset was collected from a **Robotic Spot-Welding (RSW) process**.  

- **Normal condition (180 samples):** Welding under proper conditions with a new tip.  
- **Abnormal condition (180 samples):** Welding when the welding tip is worn out.  
- Each sample contains signals from **three sensors**:  
  - **Acceleration (vibration) signal**  
  - **Voltage signal**  
  - **Current signal**  

So in total, we have **360 data samples** (180 Normal + 180 Abnormal).

### 🔎 How the code works  

- We use `pd.read_csv()` to read each dataset file from **GitHub** (https://github.com/ljwg3000/UNT_MEEN).  
- The code below loads them as variables named **Normal_1 ~ Normal_180** and **Abnormal_1 ~ Abnormal_180**.  
- Each variable contains the 3 sensor signals of one welding attempt.  

In [ ]:
NoOfData = 180  # 180 Data for each robotic spot-welding condition (Normal, Abnormal)

for i in range(NoOfData):

    temp_path1 = f'https://github.com/ljwg3000/UNT_MEEN/blob/main/AI_tutorial/Dataset/Normal_{i+1}?raw=true'   # File path of normal dataset
    temp_path2 = f'https://github.com/ljwg3000/UNT_MEEN/blob/main/AI_tutorial/Dataset/Abnormal_{i+1}?raw=true' # File path of abnormal dataset

    exec(f"Normal_{i+1}   = pd.read_csv(temp_path1 , sep=',' , header=None)")
    exec(f"Abnormal_{i+1} = pd.read_csv(temp_path2 , sep=',' , header=None)")

### 📊 Example: Plotting Normal_1  

As an example, the code below shows the signals from **Normal_1**:  

- **Acceleration signal** (red)  
- **Voltage signal** (green)  
- **Current signal** (blue)  

This helps us **visualize the raw sensor data** before doing any AI analysis.  

In [ ]:
Data = Normal_1

plt.figure(figsize=(12,8))

plt.subplot(3,1,1) # Acceleration signal
plt.plot(Data.iloc[:,0] , Data.iloc[:,1], color='r')
plt.ylabel('Acceleration (g)', fontsize=12, color='r')
plt.grid()

plt.subplot(3,1,2) # Voltage signal
plt.plot(Data.iloc[:,0] , Data.iloc[:,2], color='g')
plt.ylabel('Voltage (V)', fontsize=12, color='g')
plt.grid()

plt.subplot(3,1,3) # Current signal
plt.plot(Data.iloc[:,0] , Data.iloc[:,3], color=[0,0,1])
plt.ylabel('Current (kA)',fontsize=12, color='b')
plt.xlabel('time (s)', fontsize=12)
plt.grid()

plt.show()

.

.

.

# 🧩 2. Feature Extraction

So far, we looked at **raw sensor signals** (acceleration, voltage, current).  
But instead of using the raw signals directly, AI models usually perform better if we extract **features** —  
simple numbers that summarize each signal.  

### 📌 What are features?  
A **feature** is like a "summary number" that represents an important characteristic of the signal.  
For each sensor signal, we calculate **10 features**:  

- **Max / Min values** (highest and lowest points)  
- **Mean** (average)  
- **RMS (Root Mean Square):** like a measure of the signal’s overall strength  
- **Variance:** how spread out the values are  
- **Skewness & Kurtosis:** describe the shape of the signal distribution  
- **Crest factor, Shape factor, Impulse factor:** measures often used in vibration and fault detection  

Since we have **3 sensors**, each data sample will have **30 features (3 × 10)**.  

### **Defining a function**

One commonly used feature is the Root Mean Square (RMS) value.

The **RMS** value of a signal " $x = [x_1, x_2, \dots, x_N]$ " is defined as:



$RMS(x) = \sqrt{ \frac{1}{N} \sum_{i=1}^{N} x_i^2 }$


where:  
- $N$: number of samples  
- $x_i$: the individual data points in the signal  

👉 In words: *square the values, take the mean, then take the square root.*

.

---

We can define a simple function to calculate RMS in Python as following:

In [ ]:
# Define RMS function
def rms(x):
    return np.sqrt(np.mean(x**2))

### ⚙️ Step 2-1: Setting up empty feature arrays  
- The code first creates empty arrays (`Feature_Normal`, `Feature_Abnormal`).  
- These are like empty tables that will later be filled with the calculated features.  

In [ ]:
NoOfSensor  = 3    # 3 Sensor signals: Acceleration, Voltage, Current
NoOfFeature = 10   # 10 Feature types: Max, Min, Mean, RMS, Variance, Skewness, Kurtosis, Crest factor, Shape factor, Impulse factor

# Create empty(0) arrays for normal/abnormal feature dataset (time domain)
Feature_Normal   = np.zeros((NoOfSensor*NoOfFeature , NoOfData))
Feature_Abnormal = np.zeros((NoOfSensor*NoOfFeature , NoOfData))

print(Feature_Normal.shape)
print(Feature_Abnormal.shape)

Feature_Normal

### ⚙️ Step 2-2: Extracting features from each dataset  
- The code loops through all **Normal** and **Abnormal** data samples.  
- For each sample, and for each of the 3 sensor signals:  
  - Calculate **10 features** (max, min, mean, RMS, etc.)  
  - Store them in the corresponding array.  
- This process transforms long raw signals into short, meaningful numbers.  

In [ ]:
for i in range(NoOfData):

    # Declare temporary data
    exec(f"temp_data1 = Normal_{i+1}")
    exec(f"temp_data2 = Abnormal_{i+1}")

    # Time domain feature extraction
    for j in range(NoOfSensor):

        # Normal features
        Feature_Normal[NoOfFeature*j+0, i] = np.max(temp_data1.iloc[:,j+1])
        Feature_Normal[NoOfFeature*j+1, i] = np.min(temp_data1.iloc[:,j+1])
        Feature_Normal[NoOfFeature*j+2, i] = np.mean(temp_data1.iloc[:,j+1])
        Feature_Normal[NoOfFeature*j+3, i] = rms(temp_data1.iloc[:,j+1])
        Feature_Normal[NoOfFeature*j+4, i] = np.var(temp_data1.iloc[:,j+1])
        Feature_Normal[NoOfFeature*j+5, i] = sp.skew(temp_data1.iloc[:,j+1])
        Feature_Normal[NoOfFeature*j+6, i] = sp.kurtosis(temp_data1.iloc[:,j+1])
        Feature_Normal[NoOfFeature*j+7, i] = np.max(temp_data1.iloc[:,j+1])/rms(temp_data1.iloc[:,j+1])
        Feature_Normal[NoOfFeature*j+8, i] = rms(temp_data1.iloc[:,j+1])/np.mean(np.abs(temp_data1.iloc[:,j+1]))
        Feature_Normal[NoOfFeature*j+9, i] = np.max(temp_data1.iloc[:,j+1])/np.mean(np.abs(temp_data1.iloc[:,j+1]))

        # Abnormal features
        Feature_Abnormal[NoOfFeature*j+0, i] = np.max(temp_data2.iloc[:,j+1])
        Feature_Abnormal[NoOfFeature*j+1, i] = np.min(temp_data2.iloc[:,j+1])
        Feature_Abnormal[NoOfFeature*j+2, i] = np.mean(temp_data2.iloc[:,j+1])
        Feature_Abnormal[NoOfFeature*j+3, i] = rms(temp_data2.iloc[:,j+1])
        Feature_Abnormal[NoOfFeature*j+4, i] = np.var(temp_data2.iloc[:,j+1])
        Feature_Abnormal[NoOfFeature*j+5, i] = sp.skew(temp_data2.iloc[:,j+1])
        Feature_Abnormal[NoOfFeature*j+6, i] = sp.kurtosis(temp_data2.iloc[:,j+1])
        Feature_Abnormal[NoOfFeature*j+7, i] = np.max(temp_data2.iloc[:,j+1])/rms(temp_data2.iloc[:,j+1])
        Feature_Abnormal[NoOfFeature*j+8, i] = rms(temp_data2.iloc[:,j+1])/np.mean(np.abs(temp_data2.iloc[:,j+1]))
        Feature_Abnormal[NoOfFeature*j+9, i] = np.max(temp_data2.iloc[:,j+1])/np.mean(np.abs(temp_data2.iloc[:,j+1]))

print(Feature_Normal.shape)
print(Feature_Abnormal.shape)

Feature_Normal

### ⚙️ Step 2-3: Combining Normal and Abnormal features  
- After extracting, we combine the two arrays into one dataset:  
  - **Normal features**  
  - **Abnormal features**  
- Now we have a single dataset (`FeatureData`) that can be used for AI training.  


In [ ]:
FeatureData = pd.DataFrame(np.concatenate([Feature_Normal, Feature_Abnormal] , axis=1))
FeatureData.shape

### 💾 Step 2-4: Saving the feature dataset  
- Finally, the dataset is saved as a `.csv` file in Google Drive.  
- This file contains all the features we will use to train and test our AI model.  

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/FeatureData.csv'
FeatureData.to_csv(path, sep=',', header=None , index=None)

.

.

.

# 📊 3. Preparing Data and Labels for Machine Learning  

Once we have extracted features, the next step is to prepare them for training a machine learning model.  
This process has several important steps:

### ⚙️ Step 3-1. Standardizing Features  
Different features may have very different scales (for example, current in thousands vs. voltage in tens).  
To make sure all features contribute equally to the learning process, we **standardize** them using `StandardScaler`.  
This rescales all features so that they have similar ranges, improving both the speed and accuracy of training.


In [ ]:
# Standardize feature values
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

FeatureData_std = StandardScaler().fit_transform(FeatureData.T)
FeatureData_std.shape

### ⚙️ Step 3-2. Splitting Training and Test Sets  
To evaluate the performance of our model, we divide the dataset into two parts:  

- **Training set (80%)**: Used to train the model.  
- **Test set (20%)**: Used to evaluate how well the model performs on unseen data.  

We use the `train_test_split` function from **scikit-learn** to randomly split the data.  
A fixed `random_state` ensures that the split is reproducible.

In [ ]:
# Number of data for each condition: 180
NormalSet   = FeatureData_std[:NoOfData , :]
AbnormalSet = FeatureData_std[NoOfData: , :]

NormalSet.shape, AbnormalSet.shape

In [ ]:
from sklearn.model_selection import train_test_split

# Designate test data ratio
TestData_Ratio = 0.2

TrainData_Nor, TestData_Nor = train_test_split(NormalSet  , test_size=TestData_Ratio, random_state=777)
TrainData_Abn, TestData_Abn = train_test_split(AbnormalSet, test_size=TestData_Ratio, random_state=777)

print(TrainData_Nor.shape, TestData_Nor.shape)
print(TrainData_Abn.shape, TestData_Abn.shape)

### ⚙️ Step 3-3. Creating Labels with One-Hot Encoding  
Machine learning models need labels that tell whether each sample is **Normal** or **Abnormal**.  
We use **one-hot encoding**, which represents categories as binary vectors:  

- `[1, 0]` → Normal  
- `[0, 1]` → Abnormal  

This format is easier for neural networks to process.  
Labels are created separately for training and test datasets using `np.zeros` and `np.ones`.

In [ ]:
TrainLabel_Nor = np.zeros((TrainData_Nor.shape[0],2))
TrainLabel_Abn = np.ones( (TrainData_Abn.shape[0],2))
TestLabel_Nor  = np.zeros((TestData_Nor.shape[0],2))
TestLabel_Abn  = np.ones( (TestData_Abn.shape[0],2))

TrainLabel_Nor[:,0] = 1  # [1,0]: Normal
TrainLabel_Abn[:,0] = 0  # [0,1]: Abnormal
TestLabel_Nor[:,0]  = 1  # [1,0]: Normal
TestLabel_Abn[:,0]  = 0  # [0,1]: Abnormal

print(TrainLabel_Nor.shape, TestLabel_Nor.shape)
print(TrainLabel_Abn.shape, TestLabel_Abn.shape)

In [ ]:
# Check a label set
TestLabel_Nor

### ⚙️ Step 3-4. Combining Data and Labels  
Finally, we merge the Normal and Abnormal sets together:  

- `TrainData` and `TestData` hold all feature values.  
- `TrainLabel` and `TestLabel` hold the corresponding one-hot encoded labels.  

Now, both the input features and target labels are ready for training and evaluating the machine learning model.  


In [ ]:
TrainData  = np.concatenate([TrainData_Nor , TrainData_Abn ], axis=0)
TestData   = np.concatenate([TestData_Nor  , TestData_Abn  ], axis=0)
TrainLabel = np.concatenate([TrainLabel_Nor, TrainLabel_Abn], axis=0)
TestLabel  = np.concatenate([TestLabel_Nor , TestLabel_Abn ], axis=0)

print(TrainData.shape,  TestData.shape)
print(TrainLabel.shape, TestLabel.shape)

.

.

.

.

.

# 🤖 4. AI Modeling with MLP (Multi-Layer Perceptron)

Now that we have prepared our dataset, we move to the **AI modeling stage**.  
Here we build, train, and evaluate a simple **neural network model** using TensorFlow and Keras.



### ⚙️ Step 4-1. Importing TensorFlow
- We first import **TensorFlow**, one of the most widely used deep learning frameworks.  
- TensorFlow (with Keras) provides tools to easily build and train neural networks.  
- We also check the TensorFlow version to ensure compatibility.

In [ ]:
# Import the 'Tensorflow' pakage
import tensorflow as tf
from tensorflow import keras

# Check the version of tensorflow
print(tf.__version__)

### ⚙️ Step 4-2. Setting Hyperparameters
Before training, we define **hyperparameters**, which control how the model learns:
- **learningRate**: how fast the model updates during training.  
- **noOfNeuron**: number of neurons in each hidden layer.  
- **Epoch**: how many times the model sees the entire dataset during training.  

These values affect both the **speed** and **accuracy** of learning.

In [ ]:
learningRate  = 0.0001
noOfNeuron    = 16
Epoch         = 200

### ⚙️ Step 4-3. Designing the MLP Model
We design a neural network using **Keras Sequential API**:
- **Input Layer**: matches the number of features in the dataset.  
- **Hidden Layers**: use ReLU activation to capture non-linear patterns.  
- **Output Layer**: uses Softmax activation with 2 neurons (Normal / Abnormal classification).  

The model is compiled with:
- **Optimizer**: Adam (adaptive learning optimizer).  
- **Loss Function**: Categorical Crossentropy (for classification).  
- **Metric**: Accuracy (to measure performance).


- Types of Activation Functions: https://keras.io/api/layers/activations/

- Types of Optimization Algorithms: https://keras.io/api/optimizers/

- Types of Loss Functions (for Classification)  https://keras.io/api/losses/probabilistic_losses/

In [ ]:
def MLP_model(input_data):
    keras.backend.clear_session() # clearing the Keras backend session (initiating variables)

    model = keras.Sequential()
    model.add(keras.layers.InputLayer(input_shape = (input_data.shape[1],) ))                                      # Input  Layer
    model.add(keras.layers.Dense(units = noOfNeuron, activation = keras.activations.relu,    name = 'Hidden1'))    # Hidden Layer 1
    model.add(keras.layers.Dense(units = noOfNeuron, activation = keras.activations.relu,    name = 'Hidden2'))    # Hidden Layer 2
    model.add(keras.layers.Dense(units = 2,          activation = keras.activations.softmax, name = 'Output'))     # Output Layer

    model.compile(optimizer = keras.optimizers.Adam(learning_rate = learningRate), # Optimization algorithm
                  loss = keras.losses.CategoricalCrossentropy(),                   # Loss function (objective function of Optimization)
                  metrics = ['accuracy'])                                          # Metrics to measure during the training process
    return model

In [ ]:
# Check the model architecture and the number of parameters
MLP = MLP_model(TrainData)
MLP.summary()

In [ ]:
# Check the parameter shape for each layer
for i in range(len(MLP.get_weights())):
    print(MLP.get_weights()[i].shape)

### ⚙️ Step 4-4. Training the Model
- The model is trained using the training dataset and labels.  
- The network updates its weights over multiple epochs to minimize the **loss function**.  
- Training history (loss & accuracy) is recorded for later visualization.  

In [ ]:
tf.random.set_seed(777) # Not necessarily required

# Model traning and validation
TraingHistory  = MLP.fit(TrainData, TrainLabel, epochs=Epoch, verbose=1)

### ⚙️ Step 4-5. Evaluating the Model
- After training, we test the model on the **test dataset** (data the model has never seen).  
- **Loss** shows how far predictions are from the true labels (lower is better).  
- **Accuracy** shows how many samples are correctly classified (closer to 100% is better).

In [ ]:
# Evaluation result for test data (not trained)
Loss, Accuracy = MLP.evaluate(TestData,  TestLabel, verbose=0)
Loss, Accuracy # The closer the Loss is to 0 and the closer the accuracy is to 1 (100%), the better.

### ⚙️ Step 4-6. Visualizing Training Progress
- We plot **loss and accuracy vs. epochs** to check if the model is learning properly.  
- Smooth decrease in loss and increase in accuracy → good learning.  
- If accuracy improves only on training data but not on test data → possible **overfitting**.

In [ ]:
# Check the training process (Loss, Accuracy)

fig, loss_ax = plt.subplots(figsize=(8,6))
acc_ax = loss_ax.twinx()

loss_ax.plot(TraingHistory.history['loss'], label='train loss', c = 'tab:red')
loss_ax.set_xlabel('epoch', fontsize=15)
loss_ax.set_ylabel('loss', fontsize=15)
loss_ax.legend(loc='upper left', fontsize=15)

acc_ax.plot(TraingHistory.history['accuracy'], label='train acc', c = 'tab:blue')
acc_ax.set_ylabel('accuracy', fontsize=15)
acc_ax.legend(loc='lower left', fontsize=15)

plt.show()

### ⚙️ Step 4-7. Saving and Loading the Model
- Once trained, the model is saved as a `.keras` file for later use.  
- We can reload the saved model without retraining, and use it to make predictions.

In [ ]:
MLP.save('/content/drive/MyDrive/Colab Notebooks/MLP_model.keras')

In [ ]:
LoadedModel = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/MLP_model.keras')

Loss, Accuracy = LoadedModel.evaluate(TestData, TestLabel, verbose=0)
print('[Performance of ANN model] \n')
print('Accuracy : {:.2f}%'.format(Accuracy*100))

In [ ]:
# Predicted result
Predicted = LoadedModel.predict(TestData)
pd.DataFrame(Predicted)

### ⚙️ Step 4-8. Evaluating with Confusion Matrix
- Accuracy alone may not fully describe performance.  
- A **confusion matrix** shows detailed results:
  - **True Positive (TP):** Abnormal correctly predicted as Abnormal.  
  - **True Negative (TN):** Normal correctly predicted as Normal.  
  - **False Positive (FP):** Normal incorrectly predicted as Abnormal.  
  - **False Negative (FN):** Abnormal incorrectly predicted as Normal.  

This helps us understand the model’s strengths and weaknesses in more detail.

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Convert TestLabel and Predicted into single-column vectors for evaluations
TestLabel_rev = np.argmax(TestLabel, axis=1)
Predicted_rev = np.argmax(Predicted, axis=1)

cm = confusion_matrix(TestLabel_rev, Predicted_rev)

plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap=plt.cm.Blues, cbar=False, square=True)
plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("Confusion Matrix of the MLP Model")
plt.show()